# Create the inference wrapper

In order to use the trained model, you must write a wrapper script that loads the model and feeds it with the data coming from the configured data source. This notebook helps you to write such a wrapper, and understand how it works.

To execute this notebook, you will need the training data and the saved model from the previous notebook [10-CreateModel](10-CreateModel.ipynb).

## Load the model

The wrapper script must first load the model into the memory in order to make the prediction as fast as possible.  
The filesystem layout (python scripts in `src` folder, model in the `models` folder) is reproduced when running the pipeline, so you can run the same code while experimenting.

In [ ]:
import sys
import joblib

sys.path.insert(0, "../src")
with open("../models/bsi-model.joblib", 'rb') as rpl:
    model = joblib.load(rpl)

## Load input data

Load the unpacked example training data to test experiment with the implementation.

In [ ]:
import json

from pathlib import Path

data_path = Path('../data/processed/example')

inputs = []
for json_file in data_path.glob('*.json'):
    with open(json_file) as f:
        json_data = f.read()
        inputs.append({"json_data": json_data})

## Create the wrapper

The `process_input(data: dict)` method serves as an entrypoint, and will be called for every input batch. It expects the `json_data` field to hold a batch of input data in `JSON` format.

It expects the `json_data` to contain a `measurements` field, from which the list of `ph1`, `ph2` and `ph3` values can be extracted.

In [ ]:
import numpy 

def process_input(data: dict):
    json_data = json.loads(data['json_data'])
    measurements = json_data['measurements']
    input_data = numpy.array([[[item['ph1'], item['ph2'], item['ph3']] for item in measurements]])
    prediction =  model.predict(input_data)
    output = {"prediction": prediction[0]}
    return output

# Test it with the example dataset
for i, input in enumerate(inputs):
    result = process_input(input)
    if result is not None: print(i+1, result)

# Return metrics

You can return multiple outputs, and some of those outputs can serve as custom metrics for your running model.

When returning such custom metric outputs, the int or float value must be serialized in a specific way, like in the example below.

The metrics have to be defined separately when creating a pipeline component.

In [ ]:
import json
import numpy

def process_input(data: dict):
    json_data = json.loads(data['json_data'])
    measurements = json_data['measurements']
    input_data = numpy.array([[[item['ph1'], item['ph2'], item['ph3']] for item in measurements]])
    prediction = model.predict(input_data)
    output = {"prediction": prediction[0]}
    # Add metrics
    features = model["preprocess"].transform(input_data)[0]
    output["model_input_min"]  = metric_output(features[0])
    output["model_input_max"]  = metric_output(features[1])
    output["model_input_mean"] = metric_output(features[2])
    return output

def metric_output(v: int or float):
    return json.dumps({"value": v})

# Test it with the example dataset
for i, input in enumerate(inputs):
    result = process_input(input)
    if result is not None: print(i+1, result)

Once the inference wrapper runs the way you desire, you can update the code in [entrypoint.py](../entrypoint.py) and to match yours, and create an edge package in notebook [30-CreatePipelinePackage](30-CreatePipelinePackage.ipynb).